In [11]:
import sys
import csv
import numpy as np
import pandas as pd
import cvxopt as co
from cvxopt import matrix
import pickle
import time
import os

In [12]:
obj = pd.read_pickle(r'D:/GitHub/COL774/Assignment 2/data/part2_data/train_data.pickle')

# We wil classify 0 and 4
class1 = 0
class2 = 4

X_full = np.reshape(obj['data'],(10000,32*32*3))
Y_full = np.reshape(obj['labels'],(10000,1))
# print(Y)

X_train = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),32*32*3))
Y_train = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),1))

X_train[0:len(np.where(Y_full==class1)[0])] = X_full[np.where(Y_full==class1)[0]]
X_train[len(np.where(Y_full==class1)[0]):] = X_full[np.where(Y_full==class2)[0]]
X_train/=255

Y_train[0:len(np.where(Y_full==class1)[0])] = 1
Y_train[len(np.where(Y_full==class1)[0]):] = -1


In [23]:
def LinearKernelSVM(x,y,C = 1.0):    
    Z = x*y
    m = x.shape[0]
    P = matrix(np.dot(Z,Z.T))
    
    q = -np.ones((m,1))
    q = matrix(q)
    
    G = np.identity(m)
    G = np.append(G, -G, axis=0)
    G = matrix(G)
    
    h = np.ones((m,1))
    h = matrix(np.append(C * h, 0 * h, axis=0))
    
    A = matrix([[y[i,0]] for i in range(m)])
    b = matrix(0.0)
    
    sol = co.solvers.qp(P, q, G, h, A, b, options={'show_progress': True})
    
    alpha = sol['x']
    
    w = np.sum(alpha*Z,axis = 0)
    eps = 1e-5
    supportVectors = [i for i in range(m) if abs(alpha[i]) > eps]
    inf = float('inf')
    mna, mxa = -inf, inf
    bSum = 0.0
    bNum = 0
    for i in range(m):
#         if abs(C - alpha[i]) < eps:
#             continue
        val = np.dot(x[i],w)
        
        if int(y[i]) == -1:
            mna = max(mna, val)
        else:
            mxa = min(mxa, val)
    b = -(mna + mxa) / 2
    return w,b,supportVectors,np.array(alpha).T

In [24]:
w,b,supportVectors,alpha = LinearKernelSVM(X_train,Y_train)

     pcost       dcost       gap    pres   dres
 0: -1.1712e+03 -1.0053e+04  5e+04  3e+00  2e-10
 1: -8.2018e+02 -7.0364e+03  1e+04  6e-01  2e-10
 2: -6.5638e+02 -3.7165e+03  5e+03  2e-01  1e-10
 3: -5.8967e+02 -1.3383e+03  9e+02  2e-02  1e-10
 4: -6.5370e+02 -9.3520e+02  3e+02  6e-03  1e-10
 5: -6.9062e+02 -8.0961e+02  1e+02  2e-03  1e-10
 6: -7.1076e+02 -7.5125e+02  4e+01  3e-04  1e-10
 7: -7.1909e+02 -7.3061e+02  1e+01  7e-05  1e-10
 8: -7.2208e+02 -7.2387e+02  2e+00  7e-06  2e-10
 9: -7.2264e+02 -7.2274e+02  1e-01  3e-07  2e-10
10: -7.2267e+02 -7.2268e+02  6e-03  2e-08  2e-10
11: -7.2268e+02 -7.2268e+02  1e-04  3e-10  2e-10
Optimal solution found.


In [25]:
print(w,b)

[ 0.40311351  0.09715971  0.99773958 ...  0.48383299 -0.04012729
  0.53011341] 0.12942423232064204


In [ ]:
# Test
obj = pd.read_pickle(r'D:/GitHub/COL774/Assignment 2/data/part2_data/test_data.pickle')
Flattened = (np.reshape(obj['data'],(obj['data'].shape[0],32*32*3)))

# We wil classify 0 and 4
class1 = 0
class2 = 4

X_full = np.reshape(obj['data'],(obj['data'].shape[0],32*32*3))
Y_full = np.reshape(obj['labels'],(obj['data'].shape[0],1))
# print(Y)

X_test = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),32*32*3))
Y_act = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),1))

X_test[0:len(np.where(Y_full==class1)[0])] = X_full[np.where(Y_full==class1)[0]]
X_test[len(np.where(Y_full==class1)[0]):] = X_full[np.where(Y_full==class2)[0]]
X_test /= 255

Y_act[0:len(np.where(Y_full==class1)[0])] = 1
Y_act[len(np.where(Y_full==class1)[0]):] = -1

Y_pred = (np.dot(X_test,w)+b).reshape((Y_pred.shape[0],1))
Y_pred[np.where(Y_pred>0)[0]] = 1
Y_pred[np.where(Y_pred<=0)[0]] = -1
acc = len(np.where(Y_pred == Y_act)[0])

print(acc/len(Y_pred))

In [ ]:
print(len(supportVectors)/X_train.shape[0])

In [ ]:
print(np.argsort(alpha[0]))
top5 = X_train[np.argsort(alpha[0])[-5:]]
print(top5.shape)

In [ ]:
top5 = top5.reshape(5,32,32,3)
%matplotlib inline
from matplotlib import pyplot as plt
for i in range(5):
    plt.imshow(top5[i])
    plt.show()

In [ ]:
print(np.max(w))
plt.imshow(w.reshape(32,32,3))

## Gaussian Kernel

In [13]:
def squaredDistanceMatrix(x, y, same=False):
    if same:
        squares = np.einsum('ij,ij->i', x, x)
        squares_fill = np.tile(squares, (squares.shape[0], 1))
        return squares_fill + squares_fill.T - 2 * np.matmul(x, x.T)
    squares_x = np.einsum('ij,ij->i', x, x)
    squares_y = np.einsum('ij,ij->i', y, y)
    return np.tile(squares_y, (squares_x.shape[0], 1)) + np.tile(squares_x, (squares_y.shape[0], 1)).T - 2 * np.matmul(x, y.T)

In [14]:
def mean_normalize(x):
    return (x-np.mean(x,axis = 0).reshape((1,x.shape[1])))/np.std(x,axis = 0).reshape((1,x.shape[1])),np.mean(x,axis = 0).reshape((1,x.shape[1])),np.std(x,axis = 0).reshape((1,x.shape[1]))

In [15]:
def GaussianKernelSVM(x,y,C = 1.0,gamma = 0.001):    
    
    m = len(y)
    squares = np.exp(-gamma * squaredDistanceMatrix(x, x, same=True))
    #squares = np.exp(-gamma * squaredDistanceMatrixOld(x, x))
    P = co.matrix(squares * np.outer(y, y))
#     print(P)
#     q = co.matrix([-1.0] * m)
#     G = co.matrix(np.vstack((-1 * np.identity(m), np.identity(m))))
#     A = co.matrix([[y[i,0]] for i in range(m)])
#     b = co.matrix([0.0])
#     h = co.matrix([0.0] * m + [C] * m)
#     sol = co.solvers.qp(P, q, G, h, A, b, options={'show_progress': True})
#     assert sol['status'] == 'optimal'
#     print(sol['x'])
    
    '''m = x.shape[0]
#     X = np.square(np.linalg.norm(x,ord = 2,axis = 1)).reshape((m,1))
# #     print(X)
#     Z = np.ones((m,m))*X
#     X = Z + Z.T - 2*np.dot(x,x.T)    
    P = np.outer(y,y)*np.exp(-gamma*squaredDistanceMatrix(x,x,True))#np.exp(-gamma*X)
#     print(P)
    P = matrix(P)'''
    q = -np.ones((m,1))
    q = matrix(q)
    
    G = np.identity(m)
    G = np.append(G, -G, axis=0)
    G = matrix(G)
    
    h = np.ones((m,1))
    h = matrix(np.append(C * h, 0 * h, axis=0))
    
    A = matrix([[y[i,0]] for i in range(m)])
    b = matrix(0.0)
    
    sol = co.solvers.qp(P, q, G, h, A, b, options={'show_progress': True})
    
    alpha = (np.array(sol['x']).T)[0]
    
    
    

    value = np.einsum('i,i,ij->j', alpha, y.reshape((m,)), squares)
    
    inf = float('inf')
    mna, mxa = inf, -inf
    bSum = 0.0
    bNum = 0
    eps = 1e-5
    supportVectors = [i for i in range(m) if abs(alpha[i]) > eps]

    mna, mxa = -inf, inf
    
    for i in range(m):
        val = value[i]
        
        if int(y[i]) == -1:
            mna = max(mna, val)
        else:
            mxa = min(mxa, val)
    b = -(mna + mxa) / 2
    return b,supportVectors,alpha

In [16]:
obj = pd.read_pickle(r'D:/GitHub/COL774/Assignment 2/data/part2_data/train_data.pickle')

# We wil classify 0 and 4
class1 = 0
class2 = 4

X_full = np.reshape(obj['data'],(10000,32*32*3))
Y_full = np.reshape(obj['labels'],(10000,1))
# print(Y)

X_train = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),32*32*3))
Y_train = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),1))

X_train[0:len(np.where(Y_full==class1)[0])] = X_full[np.where(Y_full==class1)[0]]
X_train[len(np.where(Y_full==class1)[0]):] = X_full[np.where(Y_full==class2)[0]]
X_train/=255

Y_train[0:len(np.where(Y_full==class1)[0])] = 1
Y_train[len(np.where(Y_full==class1)[0]):] = -1


In [17]:
# X_train,norm_mean,norm_std = mean_normalize(X_train)
b,supportVectors,alpha = GaussianKernelSVM(X_train,Y_train)

     pcost       dcost       gap    pres   dres
 0: -1.7464e+03 -1.1879e+04  7e+04  3e+00  2e-13
 1: -1.1900e+03 -8.3601e+03  1e+04  4e-01  3e-13
 2: -1.1405e+03 -2.2570e+03  1e+03  8e-03  2e-13
 3: -1.3235e+03 -1.8460e+03  5e+02  3e-03  2e-13
 4: -1.3909e+03 -1.7295e+03  3e+02  2e-03  2e-13
 5: -1.4389e+03 -1.6455e+03  2e+02  9e-04  2e-13
 6: -1.4730e+03 -1.5870e+03  1e+02  4e-04  2e-13
 7: -1.4923e+03 -1.5555e+03  6e+01  2e-04  2e-13
 8: -1.5008e+03 -1.5419e+03  4e+01  1e-04  2e-13
 9: -1.5126e+03 -1.5238e+03  1e+01  2e-05  2e-13
10: -1.5164e+03 -1.5186e+03  2e+00  2e-06  3e-13
11: -1.5173e+03 -1.5174e+03  1e-01  7e-08  3e-13
12: -1.5174e+03 -1.5174e+03  3e-03  2e-09  3e-13
13: -1.5174e+03 -1.5174e+03  6e-05  3e-11  3e-13
Optimal solution found.


In [ ]:
print(alpha)

In [18]:
def testGaussian(x,x_sv,y_sv,gamma,alpha,b):
    nsv = x_sv.shape[0]
    m = x.shape[0]
    E = squaredDistanceMatrix(x_sv,x)
    Y_pred = np.einsum('i,ij->j', (alpha * y_sv).reshape(nsv,), np.exp(-gamma * E)) + b# np.sum(alpha*y_sv*np.exp(-gamma*E),axis = 0)+b
    return Y_pred.T

In [19]:
# Test

obj = pd.read_pickle(r'D:/GitHub/COL774/Assignment 2/data/part2_data/test_data.pickle')

# We wil classify 0 and 4
class1 = 0
class2 = 4

X_full = np.reshape(obj['data'],(obj['data'].shape[0],32*32*3))
Y_full = np.reshape(obj['labels'],(obj['data'].shape[0],1))
# print(Y)

X_test = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0]),32*32*3))
Y_act = np.zeros((len(np.where(Y_full==class1)[0])+len(np.where(Y_full==class2)[0])))

X_test[0:len(np.where(Y_full==class1)[0])] = X_full[np.where(Y_full==class1)[0]]
X_test[len(np.where(Y_full==class1)[0]):] = X_full[np.where(Y_full==class2)[0]]
X_test /= 255
# X_test = (X_test-norm_mean)/norm_std

Y_act[0:len(np.where(Y_full==class1)[0])] = 1
Y_act[len(np.where(Y_full==class1)[0]):] = -1
Y_pred = testGaussian(X_test,X_train[supportVectors],Y_train[supportVectors],0.001,alpha[supportVectors].reshape(-1,1),b)
print(Y_pred)
Y_pred[np.where(Y_pred>=0)[0]] = 1
Y_pred[np.where(Y_pred<0)[0]] = -1
acc = len(np.where(Y_pred == Y_act)[0])

print(acc/len(Y_act))

[ 0.75839257  0.62927043  1.37626534 ... -1.53705659 -0.76709165
 -2.5010408 ]
0.856


In [20]:
print(b)

1.8896328281405006


In [ ]:
print(np.argsort(alpha))
top5 = X_train[np.argsort(alpha)[-5:]]
print(top5.shape)
top5 = top5.reshape(5,32,32,3)
%matplotlib inline
from matplotlib import pyplot as plt
for i in range(5):
    plt.imshow(top5[i])
    plt.show()

## Sklearn

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# Linear

clf =SVC(C = 1.0, kernel = 'linear', gamma=0.001)
clf.fit(X_train,Y_train.reshape(-1,))
Y_pred = (clf.predict(X_test))
acc = len(np.where(Y_pred == Y_act.reshape(-1,))[0])
print(acc/len(Y_act))

0.791


In [ ]:
print(len(clf.support_))
alpha = clf.dual_coef_[0]
alpha = alpha.reshape((-1,1))
w = np.sum(alpha*X_train[clf.support_],axis = 0)
print(w.shape)
m = X_train.shape[0]
inf = float('inf')
mna,mxa = -inf,inf
for i in range(m):

    val = np.dot(X_train[i],w)

    if int(Y_train[i]) == -1:
        mna = max(mna, val)
    else:
        mxa = min(mxa, val)
b = -(mna + mxa) / 2 
print(w,b)

In [ ]:
clf = SVC(C = 1.0,kernel = 'rbf', gamma=0.001)
clf.fit(X_train,Y_train.reshape(-1,))
Y_pred = (clf.predict(X_test))
acc = len(np.where(Y_pred == Y_act.reshape(-1,))[0])
print(acc/len(Y_act))

In [ ]:
print(len(clf.support_))
gamma = 0.001
m = X_train.shape[0]
alpha = np.zeros(m,)
alpha[clf.support_] = clf.dual_coef_[0]
alpha = np.abs(alpha)

# alpha = clf.dual_coef_[0]
# alpha = alpha.reshape((-1,))
squares = np.exp(-gamma * squaredDistanceMatrix(X_train, X_train, same=True))
value = np.einsum('i,i,ij->j', alpha, Y_train.reshape(m,), squares)
inf = float('inf')
mna,mxa = -inf,inf
j = 0
for i in range(m):

    val = value[i]

    if int(Y_train[i,0]) == -1:
        mna = max(mna, val)
    else:
        mxa = min(mxa, val)
b = -(mna + mxa) / 2 
print(b)